# Run Test Method

In [19]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Modelo

In [20]:
from __future__ import annotations
import ast, sys
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Optional

@dataclass(frozen=True)
class CodeWarning:
    message: str
    lineno: int
    col: int
    end_lineno: Optional[int] = None
    end_col: Optional[int] = None

class DetectionRule:
    name: str = "Generic Detector"
    def search(self, src: str) -> List[CodeWarning]:
        raise NotImplementedError

class Patcher:
    name: str = "Generic Patcher"
    def patch(self, tree: ast.AST) -> ast.AST:
        raise NotImplementedError

class DCCLinter:
    def __init__(self, detectors: Iterable[DetectionRule], patchers: Iterable[Patcher]):
        self.detectors = detectors
        self.patchers = patchers

    def search(self, src: str) -> List[CodeWarning]:
        out: List[CodeWarning] = []
        for d in self.detectors:
            out.extend(d.search(src))
        return sorted(out, key=lambda w: (w.lineno, w.col))

    def patch(self, src: str) -> str:
        tree = ast.parse(src)
        for p in self.patchers:
          tree = p.patch(tree)
        return ast.unparse(tree)

# None Compares

## Detector

In [21]:
class NoneCompareDetectionRule(DetectionRule):
    name = "Detect ==/!= None"

    def search(self, src: str) -> List[CodeWarning]:
        # Detecta comparaciones con None usando == o !=.
        tree = ast.parse(src)
        warnings: List[CodeWarning] = []
        for node in ast.walk(tree):
            if isinstance(node, ast.Compare) and node.ops and node.comparators:
                op, comp = node.ops[0], node.comparators[0]
                if isinstance(op, (ast.Eq, ast.NotEq)) and isinstance(comp, ast.Constant) and comp.value is None:
                    warnings.append(CodeWarning("Use 'is' o 'is not' para comparaciones con None", node.lineno, node.col_offset))
        return warnings

## Patcher

In [22]:
class NodeComparisonPatcher(Patcher):
    name: str = "None Comparison Patcher"

    def patch(self, tree: ast.AST) -> ast.AST:
        # Convierte ==/!= None a is/is not None.
        for node in ast.walk(tree):
            if isinstance(node, ast.Compare) and node.ops and node.comparators:
                op, comp = node.ops[0], node.comparators[0]
                if isinstance(comp, ast.Constant) and comp.value is None and isinstance(op, (ast.Eq, ast.NotEq)):
                    node.ops[0] = ast.Is() if isinstance(op, ast.Eq) else ast.IsNot()
        return tree

## Tests

In [23]:
import unittest

class TestNoneCompare(unittest.TestCase):
    def setUp(self):
        self.linter = DCCLinter([NoneCompareDetectionRule()],[NodeComparisonPatcher()])

    def test_search(self):
        s = "a=None\nif a == None:\n  pass\nif a != None:\n  pass\n"
        warnings = self.linter.search(s)
        self.assertEqual(len(warnings), 2)

    def test_patch(self):
        s = "a=None\nif a == None:\n  pass\n"
        out = self.linter.patch(s)
        self.assertIn("if a is None:", out)

    # Tests extra
    def test_patch_not_equal(self):
        s = "a=None\nif a != None:\n  pass\n"
        out = self.linter.patch(s)
        self.assertIn("if a is not None:", out)

    def test_skip_when_already_is(self):
        s = "a=None\nif a is None:\n  pass\nif a is not None:\n  pass\n"
        warnings = self.linter.search(s)
        self.assertEqual(len(warnings), 0)
        out = self.linter.patch(s)
        self.assertIn("if a is None:", out)
        self.assertIn("if a is not None:", out)

    def test_detect_many_occurrences(self):
        s = (
            "a=None\n"
            "if a == None: pass\n"
            "while a != None: break\n"
            "xs = list(filter(lambda z: z != None, [1,None,2]))\n"
        )
        warnings = self.linter.search(s)
        self.assertGreaterEqual(len(warnings), 3)

In [24]:
runTests(TestNoneCompare)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK
....
----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK


## Real World Code

In [25]:
def find_cell(comment):
    """
    Busca la primera celda en el cuaderno actual cuya primera línea
    comience con el comentario dado (por ejemplo, "# Case Study 1").

    Parámetros:
        comment (str): El comentario que se quiere buscar.

    Retorna:
        str | None: El código fuente de la celda encontrada, o None si no existe.
    """
    for cell_source in In:  # `In` contiene el código fuente de todas las celdas ejecutadas
        lines = cell_source.strip().splitlines()
        if lines and lines[0].strip().startswith(comment):  # revisa la primera línea
            return cell_source
    return None

In [26]:
# Case Study 1
def find_first_even(nums):
    if nums == None:  # bad pattern
        return None
    for n in nums:
        if n % 2 == 0:
            return n
    return None

def safe_divide(a, b):
    if b != None and b != 0:  # bad pattern
        return a / b
    return None

def get_value_or_default(d, key, default=None):
    val = d.get(key)
    if val == None:  # bad pattern
        return default
    return val


def nested_check(x):
    # None en if anidado
    if x is not None:
        if x != None:  # patrón incorrecto
            return x
    return None

# None en condición de while
cnt = 0
item = 0
while item != None and cnt < 1:  # patrón incorrecto
    cnt += 1
    item = None

# None dentro de filter/lambda
clean = list(filter(lambda t: t != None, [1, None, 2, None, 3]))  # patrón incorrecto

In [27]:
linter = DCCLinter([NoneCompareDetectionRule()],[NodeComparisonPatcher()])

bad_code =  find_cell("# Case Study 1")
warnings = linter.search(bad_code)
print("-= Warnings =-")
for w in warnings:
    print(w)
good_code = linter.patch(bad_code)
print("-= Fixed Code =-")
print(good_code)

-= Warnings =-
CodeWarning(message="Use 'is' o 'is not' para comparaciones con None", lineno=4, col=7, end_lineno=None, end_col=None)
CodeWarning(message="Use 'is' o 'is not' para comparaciones con None", lineno=12, col=7, end_lineno=None, end_col=None)
CodeWarning(message="Use 'is' o 'is not' para comparaciones con None", lineno=18, col=7, end_lineno=None, end_col=None)
CodeWarning(message="Use 'is' o 'is not' para comparaciones con None", lineno=27, col=11, end_lineno=None, end_col=None)
CodeWarning(message="Use 'is' o 'is not' para comparaciones con None", lineno=34, col=6, end_lineno=None, end_col=None)
CodeWarning(message="Use 'is' o 'is not' para comparaciones con None", lineno=39, col=30, end_lineno=None, end_col=None)
-= Fixed Code =-
def find_first_even(nums):
    if nums is None:
        return None
    for n in nums:
        if n % 2 == 0:
            return n
    return None

def safe_divide(a, b):
    if b is not None and b != 0:
        return a / b
    return None

def g